In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch.autograd as autograd
import numpy as np 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
    def lengthWordToIndex(self):
        return len(self.word2index)

In [0]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [0]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/drive/My Drive/translate/question_answer.txt', encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [0]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [16]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'gar', True)

print(random.choice(pairs))

Reading lines...
Read 699281 sentence pairs
Trimmed to 12879 sentence pairs
Counting words...
Counted words:
gar 80
eng 81
['how are people there ?', 'they re friendly and hospitable .']


In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=10):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)

        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [14]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
encoder1.load_state_dict(torch.load('/content/drive/My Drive/translate/encoder_qna2.dict'))

RuntimeError: ignored

In [0]:
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
attn_decoder1.load_state_dict(torch.load('/content/drive/My Drive/translate/decoder_qan2.dict'))


<All keys matched successfully>

In [0]:
class Block(nn.Module):

    def __init__(self, block_dim):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(block_dim, block_dim),
            nn.ReLU(True),
            nn.Linear(block_dim, block_dim),
        )

    def forward(self, x):
        return self.net(x) + x


class Generator(nn.Module):

    def __init__(self, n_layers, block_dim):
        super().__init__()

        self.net = nn.Sequential(
            *[Block(block_dim) for _ in range(n_layers)]
        )

    def forward(self, x):
        return self.net(x)

In [0]:
def testGan(encoder, decoder, sentence, max_length=10):
    with torch.no_grad():
        # input_tensor = tensorFromSentence(input_lang, sentence)
        # print("input tensor",input_tensor.shape)
        # print(input_tensor)
        # input_length = input_tensor.size()[0]
        # print("input tensor",input_length)
        # encoder_hidden = encoder.initHidden()
        # print("encoder_hidden",encoder_hidden.shape)
        # encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        # print("encoder_outputs",encoder_outputs.shape)

        # for ei in range(input_length):
        #     encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        #     #print(encoder_output[0, 0].shape)
        #     encoder_outputs[ei] += encoder_output[0, 0]
        #     print(encoder_outputs)

          

        latent_dim = 256
        n_layers = 20
        block_dim = 256



      
        input_length = sentence.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        #encoder_outputs[0: input_length,:] = sentence
        generator = Generator(n_layers, block_dim)
        for ei in range(input_length):
            noise = torch.from_numpy(np.random.randint(-1, 1, (10, 256))).float()
            z = generator(noise)
            #encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            #print(encoder_output[0, 0].shape)
            encoder_outputs[ei] += z[0, 0]
            #print(encoder_outputs)



        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)
        # print("decoder_input",decoder_input.shape)
        # print("decoder_hidden",decoder_hidden.shape)
        # print("encoder_outputs",encoder_outputs.shape)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if ((topi.item() == EOS_token) or (topi.item() == '.')):
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [0]:
from numpy.random import randint

def evaluateGAN(encoder, decoder, n=200):
    latent_dim = 256
    n_layers = 20
    block_dim = 256

    for i in range(n):
        generator = Generator(n_layers, block_dim)
        generator.eval()
        generator.load_state_dict(torch.load('/content/drive/My Drive/translate/generator_qna2.th', map_location='cpu'))
        input_tensor = [[85],[86],[109],[1362],[2],[6],[1]]
        input_tensor = np.asarray(input_tensor)
        noise = torch.from_numpy(np.random.randint(1, 4434, (10, latent_dim)))
        #noise = torch.from_numpy(input_tensor).float()
        #z = generator(noise)
        #print("Z shape=",z.shape)
        #print("z=", z)
        pair = random.choice(pairs)
        #print('>', pair[0])
        #print('=', pair[1])
        output_words, attentions = testGan(encoder, decoder, noise)
        #output_words, attentions = testGan(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        #print('')

In [0]:
evaluateGAN(encoder1,attn_decoder1)


< i . <EOS>
< i m about to do . <EOS>
< you re making me me . . . <EOS>
< you re making up to the . . <EOS>
< you re kidding . <EOS>
< you re kidding to be in . <EOS>
< we . . . <EOS>
< you re up to . <EOS>
< you . <EOS>
< i m thinking . <EOS>
< i . . . . . . . . .
< you re making a student . . <EOS>
< we . . . . . . . . .
< you re making the whole . . . <EOS>
< you re up to be here . <EOS>
< you re making my of me . <EOS>
< i . <EOS>
< you re a my . <EOS>
< you are free . <EOS>
< you re making a . <EOS>
< i m sorry . <EOS>
< i m thinking . . . <EOS>
< i m good at the situation . <EOS>
< you re up a . <EOS>
< i m about to do it . <EOS>
< i . . . . . . . . .
< i . . . . . . . . .
< you are free . <EOS>
< i m getting it . <EOS>
< you re a student . <EOS>
< you re making a . . . . <EOS>
< you . <EOS>
< you re making the same age . <EOS>
< you re making a . <EOS>
< . <EOS>
< you re in the same age . <EOS>
< you re under at the . . <EOS>
< you re up to be good . <EOS>
< i m coming at the ne